# Analyzing Ptychography data using pycroscopy
### Stephen Jesse, Suhas Somnath, and Chris R. Smith, 
The Center for Nanophase Materials Science and The Institute for Functional Imaging for Materials <br>
Oak Ridge National Laboratory<br>
1/19/2017

Here, we will be working with ptychography datasets acquired using a scanning transmission electron microscope (STEM). These ptychography datsets have four dimensions - two (x, y) dimensions from the position of the electron beam and each spatial pixel contains a two dimensional (u, v) image, called a ronchigram, recorded by the detector. Though the ronchigrams are typically averaged to two values (bright field, dark field), retaining the raw ronchigrams enables deeper investigation of data to reveal the existence of different phases in the material and other patterns that would not be visible in the averaged data

## Configure the notebook first

In [ ]:
# Ensure python 3 compatibility
from __future__ import division, print_function, absolute_import, unicode_literals

# Import necessary libraries:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import display
import ipywidgets as widgets
import pycroscopy as px

# set up notebook to show plots within the notebook
% matplotlib inline

## Load pycroscopy compatible ptychography dataset

For simplicity we will use a dataset that has already been transalated form its original data format into a pycroscopy compatible hierarchical data format (HDF5 or H5) file

#### H5 files:
* are like smart containers that can store matrices with data, folders to organize these datasets, images, metadata like experimental parameters, links or shortcuts to datasets, etc.
* are readily compatible with high-performance computing facilities
* scale very efficiently from few kilobytes to several terabytes
* can be read and modified using any language including Python, Matlab, C/C++, Java, Fortran, Igor Pro, etc.

In [ ]:
# Select a file to work on:
# h5_path = px.io_utils.uiGetFile('*.h5', 'pycroscopy formatted Ptychography dataset')
h5_path = r"\\nanophase\IFgroup\SPM software development\Raw_Data\Ptychography\20120212_21_GB.h5"
print('Working on:\n' + h5_path)
# Open the file
h5_file = h5py.File(h5_path, mode='r+')

## Inspect the contents of this h5 data file
The file contents are stored in a tree structure, just like files on a contemporary computer.
The data is stored as a 2D matrix (position, spectroscopic value) regardless of the dimensionality of the data.  
In the case of these 4D ptychography datasets, the data is stored as a N x P dataset where N is the number of spatial positions of the electron beam and P is the number of pixels in the detector. 

The main dataset is always accompanied by four ancillary datasets that explain the position and spectroscopic value of any given element in the dataset.
In the case of the 2d images, the positions will be arranged as row0-col0, row0-col1.... row0-colN, row1-col0....
The spectroscopic information is trivial since the data at any given pixel is just a scalar value

In [ ]:
# define a small function called 'print_tree' to look at the folder tree stucture
def print_tree(parent):
    print(parent.name)
    if isinstance(parent, h5py.Group):
        for child in parent:
            print_tree(parent[child])

print('Datasets and datagroups within the file:')
print_tree(h5_file)
 
print('\nThe main dataset:')
print(h5_file['/Measurement_000/Channel_000/Raw_Data'])
print('\nThe ancillary datasets:')
print(h5_file['/Measurement_000/Channel_000/Position_Indices'])
print(h5_file['/Measurement_000/Channel_000/Position_Values'])
print(h5_file['/Measurement_000/Channel_000/Spectroscopic_Indices'])
print(h5_file['/Measurement_000/Channel_000/Spectroscopic_Values'])

print('\nMetadata or attributes in a datagroup')
for key in h5_file['/Measurement_000'].attrs:
    print('{} : {}'.format(key, h5_file['/Measurement_000'].attrs[key]))

## Read some basic parameters for visualization

In [ ]:
# Select the dataset containing the raw data to start working with:
h5_main = px.hdf_utils.getDataSet(h5_file, 'Raw_Data')[-1]

# Read some necessary parameters:
h5_pos_inds = px.hdf_utils.getAuxData(h5_main, auxDataName=['Position_Indices'])[0]
num_rows = len(np.unique(h5_pos_inds[:, 0]))
num_cols = len(np.unique(h5_pos_inds[:, 1]))
h5_spec_inds = px.hdf_utils.getAuxData(h5_main, auxDataName=['Spectroscopic_Indices'])[0]
num_sensor_rows = len(np.unique(h5_spec_inds[0, :]))
num_sensor_cols = len(np.unique(h5_spec_inds[1, :]))

## Visualize the Raw Ronchigrams

In [ ]:
current_inds = [int(0.5*num_rows),int(0.5*num_cols)]
half_ronch = [int(0.5*num_sensor_rows), int(0.5*num_sensor_cols)]

all_ronchs, success = px.io.hdf_utils.reshape_to_Ndims(h5_main)

all_ronchs = np.transpose(all_ronchs, [0,2,1,3])

all_ronchs = all_ronchs.reshape([num_rows*num_sensor_rows, num_cols*num_sensor_cols])

current_ronch_slice = (slice(current_inds[0]*num_sensor_rows, (current_inds[0]+1)*num_sensor_rows), 
                       slice(current_inds[1]*num_sensor_cols, (current_inds[1]+1)*num_sensor_cols))

plt.imshow(all_ronchs[current_ronch_slice], cmap=px.plot_utils.cmap_jet_white_center(), origin='lower')

In [ ]:
ronch_min = np.min(all_ronchs)
ronch_max = np.max(all_ronchs)
fig, axes = plt.subplots(ncols=2, figsize=(14,7))
axes[0].hold(True)
axes[0].imshow(all_ronchs, cmap=px.plot_utils.cmap_jet_white_center(), origin='lower')
current_roch_center = [current_inds[0]*num_sensor_rows+half_ronch[0], 
                       current_inds[1]*num_sensor_cols+half_ronch[1]]
main_vert_line = axes[0].axvline(x=current_roch_center[1], color='r')
main_hor_line = axes[0].axhline(y=current_roch_center[0], color='r')
# ronch_box = axes[0].add_patch(patches.Rectangle((current_roch_center[1] - half_ronch[1], current_roch_center[0] - half_ronch[0]),
#                                           num_sensor_cols, num_sensor_rows, fill=False,
#                                           color='red', linewidth=2))
img_zoom = axes[1].imshow(all_ronchs[current_ronch_slice],cmap=px.plot_utils.cmap_jet_white_center(), 
               vmax=ronch_max, vmin=ronch_min, origin='lower')
# axes[1].axvline(x=half_ronch[0], color='k')
# axes[1].axhline(y=half_ronch[1], color='k')

def move_zoom_box(coarse_row, coarse_col):
    current_inds[0] = coarse_row
    current_inds[1] = coarse_col
    current_roch_center = [current_inds[0]*num_sensor_rows+half_ronch[0], 
                           current_inds[1]*num_sensor_cols+half_ronch[1]]
    main_vert_line.set_xdata(current_roch_center[1])
    main_hor_line.set_ydata(current_roch_center[0])
#     ronch_box.set_x(current_roch_center[1] - half_ronch[1])
#     ronch_box.set_y(current_roch_center[0] - half_ronch[0])
    current_ronch_slice = (slice(current_inds[0]*num_sensor_rows, (current_inds[0]+1)*num_sensor_rows), 
                           slice(current_inds[1]*num_sensor_cols, (current_inds[1]+1)*num_sensor_cols))
    img_zoom.set_data(all_ronchs[current_ronch_slice])
    img_zoom.set_clim(vmax=ronch_max, vmin=ronch_min)
    display(fig)

widgets.interact(move_zoom_box, coarse_row=(0, num_rows, 1), 
                 coarse_col=(0, num_cols, 1));

## Performing Singular Value Decompostion (SVD)
SVD decomposes data (arranged as position x value) into a sequence of orthogonal components arranged in descending order of variance. The first component contains the most significant trend in the data. The second component contains the next most significant trend orthogonal to all previous components (just the first component). Each component consists of the trend itself (eigenvector), the spatial variaion of this trend (eigenvalues), and the variance (statistical importance) of the component.

Here, SVD essentially compares every single ronchigram with every other ronchigram to find statistically significant trends in the dataset. Such correlation would be infeasible if the ronchigrams were averaged to bright-field and dark-field scalar values. 

In [ ]:
# First check if SVD was already computed on this dataset:
h5_svd_group = px.hdf_utils.findH5group(h5_main, 'SVD')
if len(h5_svd_group) == 0:
    print('No prior SVD results found - doing SVD now')
    h5_svd_group = px.doSVD(h5_main, num_comps=256)
else:
    print('Taking previous SVD results already present in file')
    h5_svd_group = h5_svd_group[-1]
    
h5_u = h5_svd_group['U']
h5_v = h5_svd_group['V']
h5_s = h5_svd_group['S']
num_comps = 16

## Visualize the SVD results

##### S (variance):
The plot below shows the variance or statistical significance of the SVD components. The first few components contain the most significant information while the last few components mainly contain noise. 

Note also that the plot below is a log-log plot. The importance of each subsequent component drops exponentially.

In [ ]:
# Visualize variance of the principal components
fig, axes = px.plot_utils.plotScree(h5_s, title='Variance')

#### U (Eigenvalues or loading maps):
The plot below shows the spatial distribution of each SVD component

In [ ]:
# Visualize the eigenvalues or loading maps from SVD:
loadings = np.reshape(h5_u[:, :num_comps], (num_rows, num_cols, -1))
fig, axes = px.plot_utils.plot_map_stack(loadings, num_comps=num_comps, heading='Eigenvalues',
                                         cmap=px.plot_utils.cmap_jet_white_center())

#### V (Eigenvectors)
The V dataset contains the eigenvectors for each component

In [ ]:
# Visualize the eigenvectors from SVD:
eigenvectors = np.reshape(h5_v[:num_comps], (-1, num_sensor_rows, num_sensor_cols))
eigenvectors = np.transpose(eigenvectors, (1, 2, 0))
fig, axes = px.plot_utils.plot_map_stack(eigenvectors, num_comps=num_comps, heading='Eigenvectors',
                                         cmap=px.plot_utils.cmap_jet_white_center())

## Clustering
Clustering divides data into k clusters such that the variance within each cluster is minimized. 

In principle, clustering can be perfomed on any dataset that has some spectral values (eg. - ronchgirams in the case of the raw dataset or an array of values for each SVD component) for each position. However, the computation time increases with the size of the dataset.

Here, we will be performing k-means clustering on the U matrix from SVD. 
We want a large enough number of clusters so that K-means identifies fine nuances in the data. At the same time, we want to minimize computational time by reducing the number of clusters. We recommend 32 clusters.

In [ ]:
# Attempt to find any previous computation
h5_kmeans_group = px.hdf_utils.findH5group(h5_u, 'Cluster')
if len(h5_kmeans_group) == 0:
    print('No k-Means computation found. Doing K-Means now')
    num_clusters = 32
    num_comps_for_clustering = 128
    estimator = px.Cluster(h5_u, 'KMeans', num_comps=num_comps_for_clustering, n_clusters=num_clusters)
    h5_kmeans_group = estimator.do_cluster()
else:
    print('Taking existing results of previous K-Means computation')
    h5_kmeans_group = h5_kmeans_group[-1]
    
h5_labels = h5_kmeans_group['Labels']
h5_centroids = h5_kmeans_group['Mean_Response']

# In case we take existing results, we need to get these parameters
num_clusters = h5_centroids.shape[0]
num_comps_for_clustering = h5_centroids.shape[1]

#### Visualize k-means results

In [ ]:
label_mat = np.reshape(h5_labels, (num_rows, num_cols))
fig, axis = plt.subplots(figsize=(7,7))
axis.imshow(label_mat, cmap=px.plot_utils.cmap_jet_white_center())
axis.set_title('k-Means Cluster labels');

#### Visualize the hierarchical clustering
The vertical length of the branches indicates the relative separation between neighboring clusters.

In [ ]:
e_vals = np.reshape(h5_u[:, :num_comps_for_clustering], 
                    (num_rows, num_cols, -1))
fig = px.plot_utils.plot_cluster_dendrogram(label_mat, e_vals, 
                                            num_comps_for_clustering, 
                                            num_clusters, 
                                            last=num_clusters);

### Save and close
* Save the .h5 file that we are working on by closing it. <br>
* Also, consider exporting this notebook as a notebook or an html file. <br> To do this, go to File >> Download as >> HTML
* Finally consider saving this notebook if necessary

In [ ]:
h5_file.close()